# Create EDD Study Files From Data

This notebook creates the files needed for importing a study into Experiment Data Depot (EDD).

## Inputs and outputs

### Required file to run this notebook:
   - `../data/flaviolin/DBTL1/media_descriptions.csv` - media designs for each of the wells
   
   - `../data/flaviolin/DBTL1/OD.xlsx` - production data from the plate reader


### File generated by running this notebook:
   - `edd_experiment_description.csv`
   
   - `edd_protocol.csv`
 
    
The files are stored in the user defined directory.

## Setup

Importing needed libraries:

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import openpyxl

from core import create_media_description

### User parameters

In [2]:
CYCLE = 6

user_params = {
    'media_file': f'../data/flaviolin/DBTL{CYCLE}/media_descriptions.csv',  
    'measurement_file': f'../data/flaviolin/DBTL{CYCLE}/OD.xlsx',
    'output_file_path': f'../data/flaviolin/DBTL{CYCLE}', # Folder for output files,
    'num_replicates': 3,
    'num_designs': 16,
    'protocol_name': ['OD600', 'OD340'],
    'time_point': 48,
    'part_id': 'JBx_193086',
    'media': 'MOPS',
    'culture_volume': 15,
    'well_volume': 1500,
    'shaking_speed': 800,
    'temperature': 30,
    } 


In [3]:
df = pd.read_csv(user_params['media_file'], index_col=0)
df.head()

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4
Well,,,,,,,,,,,,,,,
A1,40.0,4.0,0.025429,20.0,2.356942,3.646289,0.007856,6.680308,4.907831,15.155637,0.000093,0.000721,0.000950,0.004621,0.000744
B1,40.0,4.0,0.025429,20.0,2.356942,3.646289,0.007856,6.680308,4.907831,15.155637,0.000093,0.000721,0.000950,0.004621,0.000744
C1,40.0,4.0,0.025429,20.0,2.356942,3.646289,0.007856,6.680308,4.907831,15.155637,0.000093,0.000721,0.000950,0.004621,0.000744
D1,40.0,4.0,0.008628,20.0,0.896852,5.044414,0.017022,13.269195,2.521720,232.495019,0.000104,0.002004,0.000558,0.001644,0.000029
E1,40.0,4.0,0.008628,20.0,0.896852,5.044414,0.017022,13.269195,2.521720,232.495019,0.000104,0.002004,0.000558,0.001644,0.000029


## Create Line Description

In [4]:
df['Line Description'] = df.apply(create_media_description, axis=1)


## Create Line Names

Add metadata for media and replicates to craft Line Names as **C[.]\_W[.]1\_[.]3-R[.]** denoting cycle number, wells occupying the same design and replicate number:

Check if it's row or column order of lines:

In [5]:
column_order = True if df.index[1][0] == 'B' else False

In [6]:
reps = user_params['num_replicates']
num_media_designs = user_params['num_designs']

def linefunction_colum_order(row):
    well = row.name
    if well[0] in 'ABC':
        return f'C{CYCLE}_WA{well[1]}_C{well[1]}-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_WD{well[1]}_F{well[1]}-R{row["Replicate"]}'                                                                                              

def linefunction_row_order(row):
    well = row.name
    if int(well[1]) < 5:
        return f'C{CYCLE}_W{well[0]}1_{well[0]}4-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_W{well[0]}5_{well[0]}8-R{row["Replicate"]}'                                                                                              

    
df['Replicate'] = [i+1 for _ in range(num_media_designs) for i in range(reps)]

if column_order:
    df['Line Name'] = df.apply(linefunction_colum_order, axis=1)
else:
    df['Line Name'] = df.apply(linefunction_row_order, axis=1)

In [7]:
df.head(2)

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Line Description,Replicate,Line Name
Well,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.025429,20.0,2.356942,3.646289,0.007856,6.680308,4.907831,15.155637,0.000093,0.000721,0.00095,0.004621,0.000744,"MOPS: 40.000000, Tricine: 4.000000, H3BO3: 0.0...",1,C6_WA1_C1-R1
B1,40.0,4.0,0.025429,20.0,2.356942,3.646289,0.007856,6.680308,4.907831,15.155637,0.000093,0.000721,0.00095,0.004621,0.000744,"MOPS: 40.000000, Tricine: 4.000000, H3BO3: 0.0...",2,C6_WA1_C1-R2


## Process measurement files

Read measurements file:

In [8]:
df_600 = pd.read_excel(user_params['measurement_file'], sheet_name='600', index_col=0)
df_340 = pd.read_excel(user_params['measurement_file'], sheet_name='340', index_col=0)

df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.1693,0.1071,0.1383,0.1489,0.1347,0.1343,0.1156,0.1655,0.0356,0.0462,0.0462,0.0458
B,0.1107,0.0995,0.1303,0.1251,0.1271,0.1049,0.1069,0.1155,0.0467,0.0466,0.0463,0.0457
C,0.1509,0.1373,0.1631,0.1782,0.1241,0.1639,0.1496,0.2049,0.0465,0.0467,0.0464,0.0456
D,0.1586,0.1210,0.1075,0.0934,0.0960,0.1109,0.1029,0.1420,0.0464,0.0464,0.0461,0.0458
E,0.1480,0.1082,0.0968,0.0890,0.0950,0.1088,0.0977,0.1336,0.0464,0.0462,0.0461,0.0455
F,0.1732,0.1294,0.1028,0.0870,0.1049,0.1121,0.0780,0.1631,0.0466,0.0461,0.0461,0.0457
G,0.0460,0.0463,0.0471,0.0467,0.0464,0.0461,0.0460,0.0467,0.0467,0.0463,0.0460,0.0455
H,0.0461,0.0462,0.0459,0.0459,0.0458,0.0457,0.0457,0.0458,0.0461,0.0462,0.0456,0.0453


In [9]:
df_340

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.3455,0.5353,0.5246,0.3369,0.4417,0.4955,0.5102,0.4609,0.1216,0.1344,0.1330,0.1305
B,0.2598,0.4889,0.5879,0.3313,0.4701,0.3892,0.5138,0.4196,0.1388,0.1378,0.1355,0.1311
C,0.3444,0.5572,0.6004,0.3299,0.3879,0.5450,0.6001,0.4711,0.1381,0.1375,0.1358,0.1317
D,0.6169,0.6473,0.7168,0.6263,0.4674,0.3411,0.4501,0.4209,0.1392,0.1388,0.1366,0.1316
E,0.6110,0.6548,0.7077,0.6353,0.4974,0.3556,0.4446,0.4216,0.1395,0.1387,0.1369,0.1314
F,0.6347,0.6890,0.7108,0.5899,0.5156,0.3265,0.4520,0.4174,0.1386,0.1387,0.1365,0.1319
G,0.1318,0.1354,0.1379,0.1389,0.1389,0.1371,0.1373,0.1391,0.1389,0.1377,0.1402,0.1313
H,0.1316,0.1340,0.1351,0.1351,0.1348,0.1339,0.1344,0.1354,0.1348,0.1348,0.1336,0.1311


### Process OD600

Normalize the data to the control well A9 value (water content):

In [10]:
zero_value_600 = df_600.at['A', 9]
df_600.loc[:, df_600.columns] -= zero_value_600
df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.1337,0.0715,0.1027,0.1133,0.0991,0.0987,0.0800,0.1299,0.0000,0.0106,0.0106,0.0102
B,0.0751,0.0639,0.0947,0.0895,0.0915,0.0693,0.0713,0.0799,0.0111,0.0110,0.0107,0.0101
C,0.1153,0.1017,0.1275,0.1426,0.0885,0.1283,0.1140,0.1693,0.0109,0.0111,0.0108,0.0100
D,0.1230,0.0854,0.0719,0.0578,0.0604,0.0753,0.0673,0.1064,0.0108,0.0108,0.0105,0.0102
E,0.1124,0.0726,0.0612,0.0534,0.0594,0.0732,0.0621,0.0980,0.0108,0.0106,0.0105,0.0099
F,0.1376,0.0938,0.0672,0.0514,0.0693,0.0765,0.0424,0.1275,0.0110,0.0105,0.0105,0.0101
G,0.0104,0.0107,0.0115,0.0111,0.0108,0.0105,0.0104,0.0111,0.0111,0.0107,0.0104,0.0099
H,0.0105,0.0106,0.0103,0.0103,0.0102,0.0101,0.0101,0.0102,0.0105,0.0106,0.0100,0.0097


Set negative values to zero, multiply the values by 10 to account for 10x dilution and keep only 8 columns and 6 rows:

In [11]:
df_600[df_600 < 0] = 0
df_600.loc[:, df_600.columns] *= 10
df_600 = df_600.iloc[:6,:8]
df_600

,1,2,3,4,5,6,7,8
A,1.337,0.715,1.027,1.133,0.991,0.987,0.800,1.299
B,0.751,0.639,0.947,0.895,0.915,0.693,0.713,0.799
C,1.153,1.017,1.275,1.426,0.885,1.283,1.140,1.693
D,1.230,0.854,0.719,0.578,0.604,0.753,0.673,1.064
E,1.124,0.726,0.612,0.534,0.594,0.732,0.621,0.980
F,1.376,0.938,0.672,0.514,0.693,0.765,0.424,1.275


### Process OD340

Do the same for OD340, except of multiplication by 10, as the OD340 measurements were taken from non-diluted samples:

In [12]:
zero_value_340 = df_340.at['A', 9]
df_340.loc[:, df_340.columns] -= zero_value_340
df_340[df_340 < 0] = 0
df_340 = df_340.iloc[:6,:8]
df_340

,1,2,3,4,5,6,7,8
A,0.2239,0.4137,0.4030,0.2153,0.3201,0.3739,0.3886,0.3393
B,0.1382,0.3673,0.4663,0.2097,0.3485,0.2676,0.3922,0.2980
C,0.2228,0.4356,0.4788,0.2083,0.2663,0.4234,0.4785,0.3495
D,0.4953,0.5257,0.5952,0.5047,0.3458,0.2195,0.3285,0.2993
E,0.4894,0.5332,0.5861,0.5137,0.3758,0.2340,0.3230,0.3000
F,0.5131,0.5674,0.5892,0.4683,0.3940,0.2049,0.3304,0.2958


Transform wide to long format, matching the order of wells in index to the one from `df`:

In [13]:
if column_order:
    indfcn = lambda x: f'{x.name}{int(x["variable"])}'
    df_600 = df_600.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.melt(ignore_index=False, value_name='OD340')
else:
    indfcn = lambda x: f'{x["variable"]}{(x.name)}'
    df_600 = df_600.T.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.T.melt(ignore_index=False, value_name='OD340')

df_600['Well'] = df_600.apply(indfcn, axis=1)
df_340['Well'] = df_340.apply(indfcn, axis=1)

df_600.index = df_600['Well']
df_340.index = df_340['Well']

df_600.drop(columns=['variable', 'Well'], inplace=True)
df_340.drop(columns=['variable', 'Well'], inplace=True)

df_600.head()

,OD600
Well,
A1,1.337
B1,0.751
C1,1.153
D1,1.230
E1,1.124


Add measurements to the main dataframe:

In [14]:
measurOD600 = user_params['protocol_name'][0]
measurOD340 = user_params['protocol_name'][1]
df[measurOD600] = df_600
df[measurOD340] = df_340
df.head(2)


,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Line Description,Replicate,Line Name,OD600,OD340
Well,,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.025429,20.0,2.356942,3.646289,0.007856,6.680308,4.907831,15.155637,0.000093,0.000721,0.00095,0.004621,0.000744,"MOPS: 40.000000, Tricine: 4.000000, H3BO3: 0.0...",1,C6_WA1_C1-R1,1.337,0.2239
B1,40.0,4.0,0.025429,20.0,2.356942,3.646289,0.007856,6.680308,4.907831,15.155637,0.000093,0.000721,0.00095,0.004621,0.000744,"MOPS: 40.000000, Tricine: 4.000000, H3BO3: 0.0...",2,C6_WA1_C1-R2,0.751,0.1382


## Create EDD Experiment Description File

In [15]:
df['Media'] = user_params['media']
df['Part ID'] = user_params['part_id']
df['Culture Volume'] = user_params['culture_volume']
df['Flask Volume'] = user_params['well_volume']
df['Growth Temperature'] = user_params['temperature']
df['Shaking speed'] = user_params['shaking_speed']
# df['Starting OD'] =
# df['Replicate Count'] = 24


In [16]:
# Invalid columns for now in EDD
# df['Humidity[%]'] = user_params['humidity']
# df['Plate'] = user_params['plate']

In [17]:
exp_descr_file = f'{user_params["output_file_path"]}/edd_experiment_description.xlsx'
df[['Line Name',
    'Line Description',
    'Part ID',
    'Media',
    'Culture Volume',
    'Flask Volume',
    'Growth Temperature',
    'Shaking speed',
]].to_excel(exp_descr_file, index=False)

## Create EDD Measurement File

OD600

In [18]:
measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD600}.xlsx'
df['Measurement Type'] = 'Optical Density'

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD600]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD340

In [19]:
measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD340}.xlsx'

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD340]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)